# List MLE Test

This notebook examines the loss function listmle introduced in 1390156.1390306 http://icml2008.cs.helsinki.fi/papers/167.pdf
and attempts to find a normalization factor to deal with an undesired property

In [1]:
import numpy as np
import math

In [145]:
def listmle(x):
    '''args:
        x: a list of scores for feature vectors,
        ordered by their ground truth rank
    returns:
        listMLE loss
    '''
    x_exp = np.exp(x)
    x_exp = np.flip(x_exp, axis=0)
    exp_cum_sum = np.cumsum(x_exp)
    exp_cum_sum = np.flip(exp_cum_sum, axis=0)
    return np.sum(np.log(exp_cum_sum) - x)

def normalized_listmle(x):
    '''args:
        x: a list of scores for feature vectors,
        ordered by their ground truth rank
    returns:
        normalized listMLE
    '''
    x = x -np.min(x)
    if np.max(x) != 0:
        x = x / np.max(x)
    sum_x = sum(x)
    if not sum_x:
        sum_x = 1
    # x = [val / float(sum_x) for val in x]
    # x = [sigmoid(n) for n in x]
    x_exp = np.exp(x)
    x_exp = np.flip(x_exp, axis=0)
    exp_cum_sum = np.cumsum(x_exp)
    exp_cum_sum = np.flip(exp_cum_sum, axis=0) 
    return np.sum(np.log(exp_cum_sum)  - x) / np.log(math.factorial(len(x)))

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

## Sensitivity to Score Range vs Sensitivy to correct ranking

ListMLE rewards the difference between item 1 and item N heavily, where as normalized ListMLE is less sensitive

In [146]:
print(listmle([50, 40, 30, 20, 10]))
print(normalized_listmle([50, 40, 30, 20, 10]))

0.00018160178023052254
0.7658302279538801


In [147]:
print(listmle([5, 4, 3, 2, 1]))
print(normalized_listmle([5, 4, 3, 2, 1]))

1.6129717464613917
0.7658302279538801


In [148]:
print(listmle([100, 15, 5, 4, 1]))
print(normalized_listmle([100, 15, 5, 4, 1]))

0.3752129240202766
0.8348829894795526


## Sensitivity to range of scores
ListMLE is sensitive to the range of scores, whereas normalized listMLE retruns the same score for all correctly ranked lists of uniformly distanced scores of a given length

In [149]:
print(listmle([5, 4, 3, 2, 1]))
print(normalized_listmle([5, 4, 3, 2, 1]))

1.6129717464613917
0.7658302279538801


In [150]:
print(listmle([50, 40, 30, 20, 10]))
print(normalized_listmle([50, 40, 30, 20, 10]))

0.00018160178023052254
0.7658302279538801


In [151]:
print(listmle([50, 49, 48, 47, 46]))
print(normalized_listmle([50, 49, 48, 47, 46]))

1.6129717464613904
0.7658302279538801


In [152]:
print(listmle([500, 400, 300, 200, 100]))
print(normalized_listmle([500, 400, 300, 200, 100]))

0.0
0.7658302279538801


Loss for normal listMLE decreases if the score of the first element increases, even if the rankings do not improve. This seems like it might lead to problems when used as a neural learning to rank loss funciton, as the network could minimize the loss function by maximizing the value of one item, even without improving the overall rankings.

In [153]:
print(listmle([50, 48, 49, 47, 46]))
print(normalized_listmle([50, 48, 49, 47, 46]))

2.3752118015733004
0.7893288548827587


In [154]:
print(listmle([60, 48, 49, 47, 46]))
print(normalized_listmle([60, 48, 49, 47, 46]))

1.9233233430705994
0.8286985830818072


In [155]:
print(listmle([48.5, 48, 49, 47, 46]))
print(normalized_listmle([48.5, 48, 49, 47, 46]))

3.1931891467203215
0.8070334361647374


## Sensitivity to length of list
both listMLE and normalized listMLE are sensitive to the length of the lists

In [124]:
print(listmle([3, 2, 1]))
print(normalized_listmle([3, 2, 1]))

0.7208676519626027
0.6442531347799542


In [125]:
print(listmle([5, 4, 3, 2, 1]))
print(normalized_listmle([5, 4, 3, 2, 1]))

1.6129717464613917
0.7658302279538801


In [126]:
print(listmle([20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7]))
print(normalized_listmle([20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7]))

5.737123652372659
0.8707988057569969


In [127]:
print(listmle([1, 1, 1]))
print(normalized_listmle([1, 1, 1]))

1.7917594692280547
1.0


In [128]:
print(listmle([1, 1, 1, 1]))
print(normalized_listmle([1, 1, 1, 1]))

3.1780538303479453
1.0


In [129]:
print(listmle(np.flip(np.linspace(1, 5, 4), axis=0)))
print(normalized_listmle(np.flip(np.linspace(1, 5, 4), axis=0)))


0.8225933226855302
0.7227566990100301


In [130]:
print(listmle(np.flip(np.linspace(1, 5, 5), axis=0)))
print(normalized_listmle(np.flip(np.linspace(1, 5, 5), axis=0)))

1.6129717464613917
0.7658302279538801


## Sanity Check: Functions decrease as list becomes closer to correct rank

In [131]:
print(listmle([4, 5, 3, 2, 1]))
print(normalized_listmle([4, 5, 3, 2, 1]))

2.3579645005040084
0.7856224506411679


In [132]:
print(listmle([4.5, 5, 3, 2, 1]))
print(normalized_listmle([4.5, 5, 3, 2, 1]))

1.999359629365888
0.7661220816653138


## Sensitivity to ties

In [81]:
print(listmle([0., 0., 0., 0.]))
print(normalized_listmle([0., 0., 0., 0.]))

3.1780538303479458
1.0


In [82]:

print(listmle([1., 1., 1., 1.]))
print(normalized_listmle([1., 1., 1., 1.]))

3.1780538303479453
1.0


In [83]:

print(listmle([2., 2., 2., 2.]))
print(normalized_listmle([2., 2., 2., 2.]))

3.1780538303479458
1.0


In [84]:

print(listmle([2., 2., 1., 1.]))
print(normalized_listmle([2., 2., 1., 1.]))

2.2510007625701647
0.7082953539285128


In [85]:
print(listmle([4., 3., 2., 1.]))
print(normalized_listmle([4., 3., 2., 1.]))

1.1610573505237984
0.7227566990100301


In [86]:
print(listmle([3., 3., 1., 1.]))
print(normalized_listmle([3., 3., 1., 1.]))

1.7527671383847474
0.7082953539285128


In [87]:
print(listmle([2., 1.]))
print(normalized_listmle([2., 1.]))

0.3132616875182226
0.45194108308304815


In [90]:
normalized_listmle([50000000000000,400,3,2,1])

0.8528236686889815